In [83]:
import { ChatCloudflareWorkersAI } from "@langchain/cloudflare";
import { AIMessage, BaseMessage } from "@langchain/core/messages";
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { END, MessageGraph } from "@langchain/langgraph";

const model = new ChatCloudflareWorkersAI({
  model: "@hf/thebloke/zephyr-7b-beta-awq",
  temperature: 0,
});

const graph = new MessageGraph();

graph.addNode("initial_support_agent", async (state: BaseMessage[]) => {
  const SYSTEM_TEMPLATE = `You are a secretary for LangCorp, a company that sells computers.
Be concise in your responses.
You can help customers with basic questions, but if the customer is having a billing or technical problem,
do not try to answer the question directly or gather information.
Instead, immediately transfer them to the billing or technical team by asking the user to hold for a moment.`;

  const prompt = ChatPromptTemplate.fromMessages([
    ["system", SYSTEM_TEMPLATE],
    new MessagesPlaceholder("messages"),
  ]);

  return prompt.pipe(model).invoke({ messages: state });
});

graph.addNode("billing_agent", async (state: BaseMessage[]) => {
  const SYSTEM_TEMPLATE = `You are an expert billing support specialist for LangCorp, a company that sells computers.
Help the user to the best of your ability, but be concise in your responses.
You have the ability to transfer to another agent to authorize refunds.
If you do, assume the other agent has all necessary information about the customer and their order.
You do not need to ask the user for more information.`;

  let messages = state;
  if (messages[messages.length - 1]._getType() === "ai") {
    messages = state.slice(0, -1);
  }

  const prompt = ChatPromptTemplate.fromMessages([
    ["system", SYSTEM_TEMPLATE],
    new MessagesPlaceholder("messages"),
  ]);
  return prompt.pipe(model).invoke({ messages });
});

graph.addNode("refund_tool", async (state) => {
  return new AIMessage("Refund processed!");
});

graph.addConditionalEdges("initial_support_agent", async (state) => {
  const mostRecentMessage = state[state.length - 1];
  const SYSTEM_TEMPLATE = `Your job is to detect whether a customer support representative wants to send a user to a billing team, technical team, or neither.`;
  const HUMAN_TEMPLATE = `The following text is a response from a customer support representative.
Extract whether they are routing the user to a team related to billing, technical or neither.
If they want to route the user to the billing team, respond only with the word "BILLING".
If they want to route the user to the technical team, respond only with the word "TECHNICAL".
Otherwise, respond only with the word "NEITHER".

Here is the text:

<text>
{text}
</text>

Remember, only respond with one word.`;
  const prompt = ChatPromptTemplate.fromMessages([
    ["system", SYSTEM_TEMPLATE],
    ["human", HUMAN_TEMPLATE],
  ]);
  const chain = prompt
    .pipe(model)
    .pipe(new StringOutputParser());
  const rawCategorization = await chain.invoke({ text: mostRecentMessage.content });
  return rawCategorization.toLowerCase().replace(/"/g, "");
}, {
  billing: "billing_agent",
  technical: END,
  neither: END
});

graph.addEdge("refund_tool", END);

graph.addConditionalEdges("billing_agent", async (state) => {
  const mostRecentMessage = state[state.length - 1];
  const SYSTEM_TEMPLATE = `Your job is to detect whether a billing support representative wants to authorize a refund or reject it.`;
  const HUMAN_TEMPLATE = `The following text is a response from a customer support representative.
Extract whether they are authorizing a refund or not.
If they want to refund the user, respond only with the word "REFUND".
Otherwise, respond only with the word "RESPOND".

Here is the text:

<text>
{text}
</text>

Remember, only respond with one word.`;
  const prompt = ChatPromptTemplate.fromMessages([
    ["system", SYSTEM_TEMPLATE],
    ["human", HUMAN_TEMPLATE],
  ]);
  const chain = prompt
    .pipe(model)
    .pipe(new StringOutputParser());
  const response = await chain.invoke({ text: mostRecentMessage.content });
  if (response.toLowerCase().replace(/"/g, "") === "refund") {
    return "refund";
  } else {
    return "end";
  }
}, {
  refund: "refund_tool",
  end: END
});

graph.setEntryPoint("initial_support_agent");

const runnable = graph.compile();

In [84]:
const stream = await runnable.stream(
  new HumanMessage("I've changed my mind and I want a refund for order #182818!")
);
for await (const message of stream) {
  console.log(message);
}

{
  initial_support_agent: AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "I'm sorry to hear that. Please hold while I transfer you to our billing department for further assis"... 64 more characters,
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "I'm sorry to hear that. Please hold while I transfer you to our billing department for further assis"... 64 more characters,
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
}
{
  billing_agent: AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "I'm sorry to hear that. Please provide your order number so I can initiate the refund process. Our p"... 381 more characters,
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "I'm sorry to hear that. Please provide your order number so I can initiate the refund process.